### 2.	Очистити та підготувати дані для подальшого аналізу (Silver Layer).

In [0]:
%sql
USE yahoo_data;

In [0]:
df_bronze = spark.read.format("delta").table("yahoo_data.finance_bronze")

In [0]:
# Переглянемо структуру
df_bronze.printSchema()
df_bronze.show(5)

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Dividends: double (nullable = true)
 |-- Stock_Splits: double (nullable = true)
 |-- Ticker: string (nullable = true)

+-------------------+------------------+------------------+------------------+------------------+--------+---------+------------+------+
|               Date|              Open|              High|               Low|             Close|  Volume|Dividends|Stock_Splits|Ticker|
+-------------------+------------------+------------------+------------------+------------------+--------+---------+------------+------+
|2025-01-02 05:00:00|423.90026204092493|424.43820240788705| 413.2611726539346|416.97686767578125|16896500|      0.0|         0.0|  MSFT|
|2025-01-03 05:00:00| 419.4672816518615|422.40599553232005|417.93320162542307| 421.7286071777344|1666

Обробка даних для Silver Layer
- Очищення NULL-значень
- Виправлення колонок
- Видалення дублікатів
- Форматування дат


In [0]:
from pyspark.sql.functions import col, expr
# Видалення NULL-значень
df_silver = df_bronze.na.drop()
# Приведення типу дати
df_silver = df_silver.withColumn("Date", col("Date").cast("date"))
# Видалення дублікатів
df_silver = df_silver.dropDuplicates()
# Створення нового поля – середньої ціни за день
df_silver = df_silver.withColumn("Avg_Price", expr("(High + Low) / 2"))

In [0]:
display(df_silver.head(10))

Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Ticker,Avg_Price
2025-04-24,375.01155205300716,387.7381880213899,374.5024768665421,386.59027099609375,22232300,0.0,0.0,MSFT,381.120332443966
2025-04-04,98.91000366210938,100.12999725341797,92.11000061035156,94.30999755859375,532273800,0.0,0.0,NVDA,96.11999893188477
2025-04-14,114.11000061035156,114.29000091552734,109.06999969482422,110.70999908447266,264705000,0.0,0.0,NVDA,111.68000030517578
2025-05-05,112.91000366210938,114.66999816894531,112.66000366210938,113.81999969482422,133163200,0.0,0.0,NVDA,113.66500091552734
2025-05-30,138.72000122070312,139.6199951171875,132.9199981689453,135.1300048828125,333170900,0.0,0.0,NVDA,136.2699966430664
2025-05-07,198.90915494694545,199.1788056079939,192.99690991055434,195.99298095703125,68536700,0.0,0.0,AAPL,196.08785775927413
2025-01-29,126.48836708502724,126.87833061040895,120.03896327722832,123.6886215209961,467120600,0.0,0.0,NVDA,123.45864694381864
2025-03-03,123.49864257535764,123.68861991301428,112.26967207329028,114.04950714111328,411381400,0.0,0.0,NVDA,117.97914599315229
2025-02-25,247.67521063757022,249.67259136851834,244.58926106556868,246.71646118164062,48013300,0.0,0.0,AAPL,247.1309262170435
2025-04-23,134.91000366210938,135.8699951171875,130.99000549316406,131.39999389648438,9528900,0.0,0.0,ORCL,133.43000030517578


In [0]:
# збереження в Delta Table
df_silver.write.format("delta").mode("overwrite").saveAsTable("yahoo_data.finance_silver")